In [1]:
import os,cv2,keras
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import time
import pickle

from skimage import color
from skimage import io
from keras.utils import plot_model
from keras import regularizers

from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD
from keras.layers import BatchNormalization
from keras import initializers
from keras.datasets import mnist


from kerastuner.tuners import RandomSearch
from kerastuner.tuners import BayesianOptimization
from kerastuner.engine.hyperparameters import HyperParameters

In [2]:
%load_ext autotime
RANDOM_STATE = 42

In [3]:
MEMORY_LIMIT = 4800
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=MEMORY_LIMIT)])
    except RuntimeError as e:
        print(e)

time: 81.1 ms


In [4]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

time: 99.3 ms


In [5]:
tf.test.is_gpu_available()

False

time: 76.8 ms


In [6]:
tf.test.is_built_with_cuda()

True

time: 69.7 ms


In [7]:
mnist = tf.keras.datasets.mnist

time: 61.6 ms


In [8]:
LOG_DIR = f"{int(time.time())}"

time: 70.5 ms


In [9]:
#from keras.datasets import cifar10
# load dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1))

time: 339 ms


In [10]:
X_train, X_test = X_train / 255.0, X_test / 255.0

time: 158 ms


In [11]:
def build_model(hp):

    model = Sequential()

    model.add(Conv2D(32, (5, 5),strides = [1,1], input_shape=(28, 28, 1),padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3),padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())

    model.add(Dense(10))
    model.add(Activation('softmax'))
    
    global bs
    
    bs = hp.Choice('batch_size', values=[32,48,64,96,128],default=32)

    opt = SGD(lr=hp.Choice('learning_rate', values=[1e-1,1e-2, 1e-3, 1e-4, 1e-5],default=1e-3))
    model.compile(
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                  optimizer=opt,
                  metrics=['accuracy'],
                  )
    return model

time: 2.1 ms


In [12]:
tuner = BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=50,  # how many model variations to test?
    executions_per_trial=1,  # how many trials per variation? (same model could perform differently)
    directory=LOG_DIR,
    project_name='BS_MINIST')

tuner.search_space_summary()

my_callbacks = [ tf.keras.callbacks.EarlyStopping(patience=3)]

tuner.search(x=X_train,
             y=y_train,
             verbose=1, # just slapping this here bc jupyter notebook. The console out was getting messy.
             epochs=10,
             batch_size=bs,
             callbacks=[my_callbacks],  # if you have callbacks like tensorboard, they go here.
             validation_data=(X_test,y_test))

tuner.results_summary()

with open(f"tuner_{int(time.time())}.pkl", "wb") as f:
    pickle.dump(tuner, f)

Trial 50 Complete [00h 04m 24s]
val_accuracy: 0.9901999831199646

Best val_accuracy So Far: 0.9919000267982483
Total elapsed time: 03h 45m 26s
Results summary
Results in 1627404018/BS_MINIST
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
batch_size: 32
learning_rate: 0.1
Score: 0.9919000267982483
Trial summary
Hyperparameters:
batch_size: 32
learning_rate: 0.1
Score: 0.9919000267982483
Trial summary
Hyperparameters:
batch_size: 32
learning_rate: 0.1
Score: 0.9915000200271606
Trial summary
Hyperparameters:
batch_size: 32
learning_rate: 0.1
Score: 0.9914000034332275
Trial summary
Hyperparameters:
batch_size: 32
learning_rate: 0.1
Score: 0.9914000034332275
Trial summary
Hyperparameters:
batch_size: 32
learning_rate: 0.1
Score: 0.9912999868392944
Trial summary
Hyperparameters:
batch_size: 32
learning_rate: 0.1
Score: 0.9912999868392944
Trial summary
Hyperparameters:
batch_size: 32
learning_rate: 0.1
Score: 0.9912999868392944
Trial summ